A big part of the following code was based on the Panacealab's tutorial for accesing their
open source covid-19 related tweets database. 

[Original Repo](https://github.com/thepanacealab/covid19_twitter/blob/master/COVID_19_dataset_Tutorial.ipynb)

## Required packages
***

In [13]:
from IPython.display import clear_output
'''
Required Packages
--------------------------
twarc #Twarc
tweepy # Tweepy 3.8.0
argparse #Argparse
xtract #Xtract 
wget #Wget 3.2
'''
# UNCOMMENT THE FOLLOWING LINES IF YOU DON'T HAVE THE PACKAGES INSTALLED
# %pip install twarc
# %pip install tweepy==3.8.0
# %pip install argparse
# %pip install xtract
# %pip install wget

# clear_output()

Note: you may need to restart the kernel to use updated packages.


## DataSet Import

In [1]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd

In [2]:
# dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-01-20/2021-01-20_clean-dataset.tsv.gz?raw=true" 
#@param {type:"string"}

full_dataset_path = 'datasets/full_dataset_clean.tsv.gz'
uncompressed_path = 'datasets/full_dataset_clean.tsv'
#Downloads the dataset (compressed in a GZ format)
#!wget dataset_URL -O clean-dataset.tsv.gz
# wget.download(dataset_URL, out='clean-dataset.tsv.gz')

#Unzips the dataset and gets the TSV dataset
with gzip.open(full_dataset_path, 'rb') as f_in:
    with open(uncompressed_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#Deletes the compressed GZ file
# os.unlink("clean-dataset.tsv.gz")

#Gets all possible languages from the dataset
df = pd.read_csv(uncompressed_path,sep="\t")
# lang_list = df.lang.unique()
# lang_list= sorted(np.append(lang_list,'all'))
# lang_picker = widgets.Dropdown(options=lang_list, value="all")
# lang_picker

## Filter by Language

In [9]:
# df.loc[df['lang'] == 'en'].sample(n = 1000000).to_csv('datasets/clean_en.csv')
# df.loc[df['lang'] == 'es'].sample(n = 1000000).to_csv('datasets/clean_es.csv')
# df.loc[df['lang'] == 'it'].sample(n = 1000000).to_csv('datasets/clean_it.csv')

In [12]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = lang_picker.value

#If no language specified, it will get all records from the dataset
if filtered_language == "":
  copyfile(f'clean-dataset.tsv', f'clean-dataset-filtered_{lang_picker.value}.tsv')

#If language specified, it will create another tsv file with the filtered records
else:
  filtered_tw = list()
  current_line = 1
  with open("clean-dataset.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")

    if current_line == 1:
      filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

      for line in tsvreader:
        if line[3] == filtered_language:
          filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
        current_line += 1

  print('\033[1mShowing first 5 tweets from the filtered dataset\033[0m')
  print(filtered_tw[1:(6 if len(filtered_tw) > 6 else len(filtered_tw))])

  with open(f'clean-dataset-filtered_{lang_picker.value}.tsv', 'w') as f_output:
      for item in filtered_tw:
          f_output.write(item)

Showing first 5 tweets from the filtered dataset
['1351757736208424963\t2021-01-20\t05:05:33\tit\tNULL\n', '1351757850041843715\t2021-01-20\t05:06:00\tit\tNULL\n', '1351758300082130944\t2021-01-20\t05:07:47\tit\tNULL\n', '1351758646972133378\t2021-01-20\t05:09:10\tit\tNULL\n', '1351758778790797314\t2021-01-20\t05:09:41\tit\tNULL\n']


###  Generate Twitter Authentication file

In [10]:
import json
import tweepy
from tweepy import OAuthHandler


In [8]:
# Authenticate
CONSUMER_KEY = input("Enter consumer Key") #@param {type:"string"}
CONSUMER_SECRET_KEY = input("Enter consumer Secret") #@param {type:"string"}
ACCESS_TOKEN_KEY = input("Enter access token") #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = input("Enter access token secret") #@param {type:"string"}

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)


In [10]:
# The lines below are just to test if the twitter credentials are correct
# Authenticate
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
   print ("Can't Authenticate")
else: 
    print("Authentication Succesful")

Authentication Succesful


### Hydrating the tweet

In [9]:
## Te following code downloadss a Mining tool (SMMT provided by the panacealab team to help with the hydration of tweet ids
'''
WARNING!
If you already have get_metadata.py downloaded the is no need to run this cell again
'''
from IPython.display import clear_output
import wget
import wget
# url = 'https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py'
# url = 'https://www.dropbox.com/s/u9jg4rldnnj1q4q/clean_it.csv?dl=1'
url = 'https://www.dropbox.com/s/nrqilln3boszsk2/api_keys.json?dl=1'
path = 'get_metadata.py'
path = 'clean_it.csv'
path = 'api_keys.json'
wget.download(url,out = path)


'get_metadata.py'

In [13]:
!python get_metadata.py -i "clean-dataset-filtered.tsv" -o "./datasets/hydrated_tweets" -k api_keys.json
# python3 get_metadata.py -i "clean_it.csv" -o "hydrated_tweets_it" -k api_keys.json

# clear_output()

^C


In [ ]:
!python get_metadata.py -i clean-dataset-filtered.tsv -o hydrated_tweets -k api_keys.json

In [13]:
df = pd.read_csv(f'clean-dataset-filtered_{lang_picker.value}.tsv', sep='\t',  header=0)

In [14]:
df.shape

(5427, 5)

Needed files, only uncomment and run this cell once if you don't have the scripts already in your machine

In [12]:
# wget.download('https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/parse_json_lite.py',out = 'parse_json_lite.py')
# wget.download('https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/fields.py', out = 'fields.py')

'fields.py'

### Clean and reformat hydrated files

In [17]:
!python parse_json_lite.py "./datasets/italian/hydrated_tweets_it_short.json" p

Traceback (most recent call last):
  File "c:\Users\diego\Documents\Git\NLP_202110\Stage 1\parse_json_lite.py", line 26, in <module>
    from emot.emo_unicode import UNICODE_EMO, EMOTICONS
ModuleNotFoundError: No module named 'emot'


In [16]:
# %pip install emot --upgrade
# %pip install emoji --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Italian

## Reading italian tweets

In [92]:

import json
file = open('./datasets/italian/hydrated_tweets_it_short.json', 'r')
data = []
for line in file:
    data.append(json.loads(line))

italian_raw = pd.json_normalize(data)
print(italian_raw.shape)
italian_raw

(879887, 8)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Fri Nov 06 20:32:15 +0000 2020,Il prof. #Galli è molto preoccupato da quanto ...,None,0,1,iLMeteo.it,1324811856859533313,False
1,Thu Jul 30 08:01:22 +0000 2020,Il COVID-19 coincide con un momento cruciale n...,None,1,1,Hootsuite Inc.,1288746494368124928,False
2,Tue Nov 10 15:54:51 +0000 2020,Muoviamoci #vaccinoCovid #coronavirus #COVID19...,None,0,1,Twitter for iPhone,1326191597785059329,False
3,Mon Jun 22 18:30:22 +0000 2020,@mosllerdd @Cartabellotta Si sarebbero potute ...,mosllerdd,0,7,Twitter Web App,1275134048722132993,False
4,Thu Mar 26 10:10:20 +0000 2020,@morzo6 @CNN Purtroppo sì. State a casa. Disin...,morzo6,1,2,Twitter for iPhone,1243118079091118080,False
...,...,...,...,...,...,...,...,...
879882,Fri Apr 17 09:11:49 +0000 2020,La scelta delle #CaseRiposo è stata fatta perc...,None,36,46,Twitter Web Client,1251075883982630913,False
879883,Sun Nov 15 19:29:01 +0000 2020,"E sì, proprio vero https://t.co/gVubUUdQFk",None,1,1,Twitter for Android,1328057436436115456,False
879884,Thu Jun 04 09:05:28 +0000 2020,@IamEluruSreenu anna di notification vacchindi...,IamEluruSreenu,0,0,Twitter for Android,1268468904164237319,False
879885,Tue Feb 16 08:13:17 +0000 2021,"Poco fa entra da me un cliente, ricercatore un...",None,0,0,Twitter Web App,1361589453001744384,False


### Preproccess functions

In [93]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import emoji
import re

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_urls(text):
    result = re.sub(r"http\S+", "", text)
    return(result)

def remove_twitter_urls(text):
    clean = re.sub(r"pic.twitter\S+", "",text)
    return(clean)

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

## Preproccess

Here we remove blank characters, urls information, emojis, etc 

In [94]:
def preproccess(data):
    data['text'] = data['text'].str.replace('\n','')
    print('Blank space removed')
    data['text'] = data['text'].str.replace('\r','')
    print('Special character removed')
    data['text'] = data['text'].apply(lambda x : remove_urls(x))
    print('urls removed')
    data['text'] = data['text'].apply(lambda x : remove_twitter_urls(x))
    print('Twitter urls removed')
    data['text'] = data['text'].apply(lambda x : remove_emoticons(x))
    print('Emoticons removed')
    data['text'] = data['text'].apply(lambda x : remove_emoji(x))
    print('Emojis removed')

preproccess(italian_raw)
italian_raw

Blank space removed
Special character removed
urls removed
Twitter urls removed
Emoticons removed
Emojis removed


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Fri Nov 06 20:32:15 +0000 2020,Il prof. #Galli è molto preoccupato da quanto ...,None,0,1,iLMeteo.it,1324811856859533313,False
1,Thu Jul 30 08:01:22 +0000 2020,Il COVID-19 coincide con un momento cruciale n...,None,1,1,Hootsuite Inc.,1288746494368124928,False
2,Tue Nov 10 15:54:51 +0000 2020,Muoviamoci #vaccinoCovid #coronavirus #COVID19...,None,0,1,Twitter for iPhone,1326191597785059329,False
3,Mon Jun 22 18:30:22 +0000 2020,@mosllerdd @Cartabellotta Si sarebbero potute ...,mosllerdd,0,7,Twitter Web App,1275134048722132993,False
4,Thu Mar 26 10:10:20 +0000 2020,@morzo6 @CNN Purtroppo sì. State a casa. Disin...,morzo6,1,2,Twitter for iPhone,1243118079091118080,False
...,...,...,...,...,...,...,...,...
879882,Fri Apr 17 09:11:49 +0000 2020,La scelta delle #CaseRiposo è stata fatta perc...,None,36,46,Twitter Web Client,1251075883982630913,False
879883,Sun Nov 15 19:29:01 +0000 2020,"E sì, proprio vero",None,1,1,Twitter for Android,1328057436436115456,False
879884,Thu Jun 04 09:05:28 +0000 2020,@IamEluruSreenu anna di notification vacchindi...,IamEluruSreenu,0,0,Twitter for Android,1268468904164237319,False
879885,Tue Feb 16 08:13:17 +0000 2021,"Poco fa entra da me un cliente, ricercatore un...",None,0,0,Twitter Web App,1361589453001744384,False


### Parse date time column
Here we format string column to date column

In [133]:
italian_raw['created_at'] = pd.to_datetime(italian_raw['created_at'], format='%a %b %d %H:%M:%S %z %Y',)
italian_raw['publication_date'] = pd.to_datetime(italian_raw['created_at'], unit='ms').astype(np.int64) / int(1e9) #Seconds since epoch
print(italian_raw['created_at'].dtypes)
print(italian_raw.shape)
italian_raw

datetime64[ns, UTC]
(879887, 9)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet,publication_date
0,2020-11-06 20:32:15+00:00,Il prof. #Galli è molto preoccupato da quanto ...,None,0,1,iLMeteo.it,1324811856859533313,False,1.604695e+09
1,2020-07-30 08:01:22+00:00,Il COVID-19 coincide con un momento cruciale n...,None,1,1,Hootsuite Inc.,1288746494368124928,False,1.596096e+09
2,2020-11-10 15:54:51+00:00,Muoviamoci #vaccinoCovid #coronavirus #COVID19...,None,0,1,Twitter for iPhone,1326191597785059329,False,1.605024e+09
3,2020-06-22 18:30:22+00:00,@mosllerdd @Cartabellotta Si sarebbero potute ...,mosllerdd,0,7,Twitter Web App,1275134048722132993,False,1.592851e+09
4,2020-03-26 10:10:20+00:00,@morzo6 @CNN Purtroppo sì. State a casa. Disin...,morzo6,1,2,Twitter for iPhone,1243118079091118080,False,1.585217e+09
...,...,...,...,...,...,...,...,...,...
879882,2020-04-17 09:11:49+00:00,La scelta delle #CaseRiposo è stata fatta perc...,None,36,46,Twitter Web Client,1251075883982630913,False,1.587115e+09
879883,2020-11-15 19:29:01+00:00,"E sì, proprio vero",None,1,1,Twitter for Android,1328057436436115456,False,1.605469e+09
879884,2020-06-04 09:05:28+00:00,@IamEluruSreenu anna di notification vacchindi...,IamEluruSreenu,0,0,Twitter for Android,1268468904164237319,False,1.591262e+09
879885,2021-02-16 08:13:17+00:00,"Poco fa entra da me un cliente, ricercatore un...",None,0,0,Twitter Web App,1361589453001744384,False,1.613463e+09


## Write italian json

In [153]:
italian_raw['source'] = 'twitter'
italian_raw['author'] = 'nn'
italian_raw['id'] = italian_raw['id_str']
columns = ['id', 'publication_date', 'source', 'text']
italian_raw[columns].to_json(path_or_buf='./datasets/italian/italianV1.json', orient='records', lines=True)
print('Vanilla json wrote')
# Compressedd version
italian_raw[columns].to_json(path_or_buf='./datasets/italian/italianV1.json.gz', orient='records', lines=True, compression='gzip')
print('Compressed json wrote')

Vanilla json wrote
Compressed json wrote


# Spanish

Reading spanish tweets

In [154]:
import json
file = open('./datasets/spanish/hydrated_tweets_es_short.json', 'r')
data = []
for line in file:
    data.append(json.loads(line))

spanish_raw = pd.json_normalize(data)
print(spanish_raw.shape)
spanish_raw

(886351, 8)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Fri Sep 11 12:08:19 +0000 2020,ORACIÓN DIARIA\nViernes 11 de Septiembre 2020\...,None,4,19,Twitter for Android,1304391319972790274,False
1,Wed Sep 23 17:40:08 +0000 2020,Se pudrió todo. Acá la FIFA debe castigar a la...,None,1,5,Twitter for Android,1308823479077175297,False
2,Thu Apr 30 18:39:13 +0000 2020,☣️😷 Última Publicación en la Prensaldia - 😉...,None,0,0,Instagram,1255929719893221377,False
3,Wed Mar 03 08:09:07 +0000 2021,Australia = 0 positivos por coronavirus.\n¿Vac...,None,0,0,Twitter for Android,1367024221319286784,False
4,Thu Jul 02 13:56:34 +0000 2020,coronavirus esto ya es personal https://t.co/O...,None,0,0,Twitter for Android,1278689023087849480,False
...,...,...,...,...,...,...,...,...
886346,Fri Mar 06 22:47:29 +0000 2020,La gente pillando coronavirus\nY tú sin pillar...,None,0,4,Twitter for Android,1236060862122799104,False
886347,Sat Apr 10 08:20:02 +0000 2021,"📝 El martes 27 de abril, no te pierdas el prim...",None,0,3,Hootsuite Inc.,1380797710693982208,False
886348,Tue Feb 25 14:35:22 +0000 2020,Italia registra el primer caso de coronavirus ...,None,1,5,Twitter Web Client,1232313141091946496,False
886349,Thu Apr 01 11:21:29 +0000 2021,Colombia registra 8.646 casos nuevos y 167 mue...,None,0,0,Twitter for iPhone,1377581882553069569,False


## Spanish Preproccess

Here we remove blank characters, urls information, emojis, etc 

In [155]:
preproccess(spanish_raw)
spanish_raw

Blank space removed
Special character removed
urls removed
Twitter urls removed
Emoticons removed
Emojis removed


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Fri Sep 11 12:08:19 +0000 2020,ORACIÓN DIARIAViernes 11 de Septiembre 2020#or...,None,4,19,Twitter for Android,1304391319972790274,False
1,Wed Sep 23 17:40:08 +0000 2020,Se pudrió todo. Acá la FIFA debe castigar a la...,None,1,5,Twitter for Android,1308823479077175297,False
2,Thu Apr 30 18:39:13 +0000 2020,Última Publicación en la Prensaldia - @ca...,None,0,0,Instagram,1255929719893221377,False
3,Wed Mar 03 08:09:07 +0000 2021,Australia = 0 positivos por coronavirus.¿Vacun...,None,0,0,Twitter for Android,1367024221319286784,False
4,Thu Jul 02 13:56:34 +0000 2020,coronavirus esto ya es personal,None,0,0,Twitter for Android,1278689023087849480,False
...,...,...,...,...,...,...,...,...
886346,Fri Mar 06 22:47:29 +0000 2020,La gente pillando coronavirusY tú sin pillarme...,None,0,4,Twitter for Android,1236060862122799104,False
886347,Sat Apr 10 08:20:02 +0000 2021,"El martes 27 de abril, no te pierdas el prime...",None,0,3,Hootsuite Inc.,1380797710693982208,False
886348,Tue Feb 25 14:35:22 +0000 2020,Italia registra el primer caso de coronavirus ...,None,1,5,Twitter Web Client,1232313141091946496,False
886349,Thu Apr 01 11:21:29 +0000 2021,Colombia registra 8.646 casos nuevos y 167 mue...,None,0,0,Twitter for iPhone,1377581882553069569,False


### Spanish Parse date time column
Here we format string column to date column

In [156]:
spanish_raw['created_at'] = pd.to_datetime(spanish_raw['created_at'], format='%a %b %d %H:%M:%S %z %Y',)
spanish_raw['publication_date'] = pd.to_datetime(spanish_raw['created_at'], unit='ms').astype(np.int64) / int(1e9) #Seconds since epoch
print(spanish_raw['created_at'].dtypes)
print(spanish_raw.shape)
spanish_raw

datetime64[ns, UTC]
(886351, 9)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet,publication_date
0,2020-09-11 12:08:19+00:00,ORACIÓN DIARIAViernes 11 de Septiembre 2020#or...,None,4,19,Twitter for Android,1304391319972790274,False,1.599826e+09
1,2020-09-23 17:40:08+00:00,Se pudrió todo. Acá la FIFA debe castigar a la...,None,1,5,Twitter for Android,1308823479077175297,False,1.600883e+09
2,2020-04-30 18:39:13+00:00,Última Publicación en la Prensaldia - @ca...,None,0,0,Instagram,1255929719893221377,False,1.588272e+09
3,2021-03-03 08:09:07+00:00,Australia = 0 positivos por coronavirus.¿Vacun...,None,0,0,Twitter for Android,1367024221319286784,False,1.614759e+09
4,2020-07-02 13:56:34+00:00,coronavirus esto ya es personal,None,0,0,Twitter for Android,1278689023087849480,False,1.593698e+09
...,...,...,...,...,...,...,...,...,...
886346,2020-03-06 22:47:29+00:00,La gente pillando coronavirusY tú sin pillarme...,None,0,4,Twitter for Android,1236060862122799104,False,1.583535e+09
886347,2021-04-10 08:20:02+00:00,"El martes 27 de abril, no te pierdas el prime...",None,0,3,Hootsuite Inc.,1380797710693982208,False,1.618043e+09
886348,2020-02-25 14:35:22+00:00,Italia registra el primer caso de coronavirus ...,None,1,5,Twitter Web Client,1232313141091946496,False,1.582641e+09
886349,2021-04-01 11:21:29+00:00,Colombia registra 8.646 casos nuevos y 167 mue...,None,0,0,Twitter for iPhone,1377581882553069569,False,1.617276e+09


## Write Spanish json

In [157]:
spanish_raw['source'] = 'twitter'
spanish_raw['author'] = 'nn'
spanish_raw['id'] = spanish_raw['id_str']
columns = ['id', 'publication_date', 'source', 'text']
spanish_raw[columns].to_json(path_or_buf='./datasets/spanish/spanishV1.json', orient='records', lines=True)
print('Vanilla json wrote')
# Compressedd version
spanish_raw[columns].to_json(path_or_buf='./datasets/spanish/spanishV1.json.gz', orient='records', lines=True, compression='gzip')
print('Compressed json wrote')

Vanilla json wrote
Compressed json wrote


# English
Reading english tweets

In [158]:
import json
file = open('./datasets/english/hydrated_tweets_en_short.json', 'r')
data = []
for line in file:
    data.append(json.loads(line))

english_raw = pd.json_normalize(data)
print(english_raw.shape)
english_raw

(817046, 8)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Wed Aug 19 03:42:33 +0000 2020,Info Source:\nhttps://t.co/CUnx24XUTw https://...,None,0,0,PromoRepublic,1295929115770593287,False
1,Fri Aug 21 09:18:49 +0000 2020,#PostponeJEE_NEETSept \n#ProtestAgainstExamsIn...,None,3,0,Twitter for Android,1296738518216011777,False
2,Tue Apr 21 04:14:45 +0000 2020,Coronavirus-spreader Chris Cuomo got a lecture...,None,0,4,Twitter Web Client,1252450676015198210,False
3,Sat Apr 10 00:52:03 +0000 2021,Any military member that refuses to get vaccin...,None,2,24,Twitter for Android,1380684968880406528,False
4,Mon Mar 08 16:16:03 +0000 2021,#Covid19 is staying around for a while. \n\nht...,None,0,0,Buffer,1368958702150156290,False
...,...,...,...,...,...,...,...,...
817041,Tue Jun 09 18:31:56 +0000 2020,"Children receiving free school meals, from sin...",None,2,8,Twitter Web App,1270423398120935426,False
817042,Fri Jul 17 03:38:52 +0000 2020,L.A. County reports highest number of coronavi...,None,4,5,SocialFlow,1283969389117792258,False
817043,Sat Sep 26 12:37:28 +0000 2020,Progress ?\nCovid19 Political Development http...,None,0,0,Twitter for Android,1309834472053903360,False
817044,Fri Jul 03 14:40:28 +0000 2020,@WeeklyEconPod @AyeishaTS @JCWI_UK @ZoeJardini...,NEF,2,4,TweetDeck,1279062456468492289,False


## English Preproccess

Here we remove blank characters, urls information, emojis, etc 

In [159]:
preproccess(english_raw)
english_raw

Blank space removed
Special character removed
urls removed
Twitter urls removed
Emoticons removed
Emojis removed


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet
0,Wed Aug 19 03:42:33 +0000 2020,Info Source:,None,0,0,PromoRepublic,1295929115770593287,False
1,Fri Aug 21 09:18:49 +0000 2020,#PostponeJEE_NEETSept #ProtestAgainstExamsInCO...,None,3,0,Twitter for Android,1296738518216011777,False
2,Tue Apr 21 04:14:45 +0000 2020,Coronavirus-spreader Chris Cuomo got a lecture...,None,0,4,Twitter Web Client,1252450676015198210,False
3,Sat Apr 10 00:52:03 +0000 2021,Any military member that refuses to get vaccin...,None,2,24,Twitter for Android,1380684968880406528,False
4,Mon Mar 08 16:16:03 +0000 2021,#Covid19 is staying around for a while. your ...,None,0,0,Buffer,1368958702150156290,False
...,...,...,...,...,...,...,...,...
817041,Tue Jun 09 18:31:56 +0000 2020,"Children receiving free school meals, from sin...",None,2,8,Twitter Web App,1270423398120935426,False
817042,Fri Jul 17 03:38:52 +0000 2020,L.A. County reports highest number of coronavi...,None,4,5,SocialFlow,1283969389117792258,False
817043,Sat Sep 26 12:37:28 +0000 2020,Progress ?Covid19 Political Development,None,0,0,Twitter for Android,1309834472053903360,False
817044,Fri Jul 03 14:40:28 +0000 2020,@WeeklyEconPod @AyeishaTS @JCWI_UK @ZoeJardini...,NEF,2,4,TweetDeck,1279062456468492289,False


### English Parse date time column
Here we format string column to date column

In [160]:
english_raw['created_at'] = pd.to_datetime(english_raw['created_at'], format='%a %b %d %H:%M:%S %z %Y',)
english_raw['publication_date'] = pd.to_datetime(english_raw['created_at'], unit='ms').astype(np.int64) / int(1e9) #Seconds since epoch
print(english_raw['created_at'].dtypes)
print(english_raw.shape)
english_raw

datetime64[ns, UTC]
(817046, 9)


,created_at,text,in_reply_to_screen_name,retweet_count,favorite_count,source,id_str,is_retweet,publication_date
0,2020-08-19 03:42:33+00:00,Info Source:,None,0,0,PromoRepublic,1295929115770593287,False,1.597809e+09
1,2020-08-21 09:18:49+00:00,#PostponeJEE_NEETSept #ProtestAgainstExamsInCO...,None,3,0,Twitter for Android,1296738518216011777,False,1.598002e+09
2,2020-04-21 04:14:45+00:00,Coronavirus-spreader Chris Cuomo got a lecture...,None,0,4,Twitter Web Client,1252450676015198210,False,1.587442e+09
3,2021-04-10 00:52:03+00:00,Any military member that refuses to get vaccin...,None,2,24,Twitter for Android,1380684968880406528,False,1.618016e+09
4,2021-03-08 16:16:03+00:00,#Covid19 is staying around for a while. your ...,None,0,0,Buffer,1368958702150156290,False,1.615220e+09
...,...,...,...,...,...,...,...,...,...
817041,2020-06-09 18:31:56+00:00,"Children receiving free school meals, from sin...",None,2,8,Twitter Web App,1270423398120935426,False,1.591728e+09
817042,2020-07-17 03:38:52+00:00,L.A. County reports highest number of coronavi...,None,4,5,SocialFlow,1283969389117792258,False,1.594957e+09
817043,2020-09-26 12:37:28+00:00,Progress ?Covid19 Political Development,None,0,0,Twitter for Android,1309834472053903360,False,1.601124e+09
817044,2020-07-03 14:40:28+00:00,@WeeklyEconPod @AyeishaTS @JCWI_UK @ZoeJardini...,NEF,2,4,TweetDeck,1279062456468492289,False,1.593787e+09


## Write English Json

In [161]:
english_raw['source'] = 'twitter'
english_raw['author'] = 'nn'
english_raw['id'] = english_raw['id_str']
columns = ['id', 'publication_date', 'source', 'text']
english_raw[columns].to_json(path_or_buf='./datasets/english/englishV1.json', orient='records', lines=True)
print('Vanilla json wrote')
# Compressedd version
english_raw[columns].to_json(path_or_buf='./datasets/english/englishV1.json.gz', orient='records', lines=True, compression='gzip')
print('Compressed json wrote')

Vanilla json wrote
Compressed json wrote
